In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [10]:
train_dir = 'Train'
val_dir = 'Validation'
test_dir = 'Test'

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
batch_size = 64
IMG_SHAPE = 224

In [13]:
image_gen = ImageDataGenerator(rescale=1./255,  zoom_range=0.1)

train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                                directory=train_dir,
                                                shuffle=True,
                                                target_size=(IMG_SHAPE,IMG_SHAPE),
                                                )

Found 10000 images belonging to 2 classes.


In [14]:
image_gen_val = ImageDataGenerator(rescale=1./255,)

val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
                                                directory=val_dir,
                                                target_size=(IMG_SHAPE,IMG_SHAPE),
                                                )

Found 800 images belonging to 2 classes.


In [15]:
image_gen_test = ImageDataGenerator(rescale=1./255,)

test_data_gen = image_gen_test.flow_from_directory(batch_size=batch_size,
                                                directory=test_dir,
                                                target_size=(IMG_SHAPE,IMG_SHAPE),
                                                )

Found 992 images belonging to 2 classes.


In [16]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMG_SHAPE, IMG_SHAPE, 3))

In [17]:
feature_extractor.trainable = False


In [18]:
model = tf.keras.Sequential([
  feature_extractor,
  tf.keras.layers.Dense(2,activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [19]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [20]:
epochs = 15

history = model.fit(
    train_data_gen,
    steps_per_epoch=int(np.ceil(train_data_gen.n / float(batch_size))),
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(val_data_gen.n / float(batch_size)))
)

Epoch 1/15
157/157 [==============================] - 85s 513ms/step - loss: 0.2119 - accuracy: 0.9362 - val_loss: 0.0597 - val_accuracy: 0.9862
Epoch 2/15
157/157 [==============================] - 79s 504ms/step - loss: 0.0333 - accuracy: 0.9915 - val_loss: 0.0383 - val_accuracy: 0.9925
Epoch 3/15
157/157 [==============================] - 79s 502ms/step - loss: 0.0205 - accuracy: 0.9959 - val_loss: 0.0322 - val_accuracy: 0.9937
Epoch 4/15
157/157 [==============================] - 79s 504ms/step - loss: 0.0147 - accuracy: 0.9966 - val_loss: 0.0314 - val_accuracy: 0.9937
Epoch 5/15
157/157 [==============================] - 80s 509ms/step - loss: 0.0157 - accuracy: 0.9956 - val_loss: 0.0276 - val_accuracy: 0.9937
Epoch 6/15
157/157 [==============================] - 79s 504ms/step - loss: 0.0112 - accuracy: 0.9980 - val_loss: 0.0216 - val_accuracy: 0.9950
Epoch 7/15
157/157 [==============================] - 79s 501ms/step - loss: 0.0087 - accuracy: 0.9978 - val_loss: 0.0247 - val_ac

In [21]:
model.evaluate(val_data_gen, batch_size=64, verbose=0, steps=(np.ceil(val_data_gen.n / float(batch_size))))

[0.019548997282981873, 0.9950000047683716]

In [22]:
model.evaluate(test_data_gen, batch_size=64, verbose=0, steps=(np.ceil(test_data_gen.n / float(batch_size))))

[0.01482042483985424, 0.9939516186714172]

In [23]:
model.save('model_mask.h5')

In [30]:
path = 'teste.jpeg'

In [32]:
image = tf.keras.preprocessing.image.load_img(path,target_size=(IMG_SHAPE,IMG_SHAPE))
input_arr = tf.keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr])  # Convert single image to a batch.
predictions = model.predict(input_arr)

In [57]:
classes = np.array(list(train_data_gen.class_indices.keys()))

In [58]:
classes

array(['WithMask', 'WithoutMask'], dtype='<U11')

In [59]:
classes[np.argmax(predictions)]

'WithMask'